# IMPORTS

In [15]:
# importing all the imports needed for this phase
# data manipulation
import pandas as pd
import numpy as np

# visuals
import matplotlib.pyplot as plt
import seaborn as sns

# personal import
import created_functions as cf
import env

# sk imports
from sklearn.model_selection import train_test_split

# ACQUIRE

In [16]:
# getting data from SQL
df =cf.get_telco_data()

this file exists, reading csv


In [17]:
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [18]:
df.shape

(7043, 24)

>there are 7043 rows and 24 columns

important
> gender
> tenure
> senior
> monthly

unimportant
> payment_type_id
> internet_service_type
> contract_type_id
> customer_id ( theres already an index)  

# PREPARE

In [19]:
df = cf.prep_telco(df)

In [20]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


### changes to data
this  function takes in the telco database and cleans it
#### deletes
    payment type id
    internet service id
    contract type id
    customer id
turns total charges into a float by deleting empty spaces

In [26]:
#combining the add on columns into 1
#adding 1 for addons and 0 for no addons
df['has_add_ons'] = df[['online_security'
    , 'online_backup', 'device_protection', 'tech_support']].apply(
    lambda row: 1 if any(x == 'Yes' for x in row) else 0, 
    axis=1
)

### data splitting

In [27]:
# splitting the data
train, validate, test = cf.splitting_data(df, 'churn')

In [28]:
# checking out new variable
train.shape, validate.shape, test.shape

((4219, 21), (1406, 21), (1407, 21))

## PREPROCESS
